In [256]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [257]:
inbound = pd.read_csv('../Data/Inbound.csv')
outbound = pd.read_csv('../Data/outbound.csv')
inventory = pd.read_csv('../Data/inventory.csv')
operation = pd.read_csv('../Data/operationCost.csv')
material = pd.read_csv('../Data/materialMaster.csv')

In [258]:
inventory.head()

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
0,12/31/2023,CHINA-WAREHOUSE,MAT-0045,SCRAP,164,KG,211,CNY
1,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6024,0,KG,87666,CNY
2,12/31/2023,CHINA-WAREHOUSE,MAT-0193,5755,70720,KG,414419,CNY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY
4,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6734,2720,KG,15939,CNY


In [259]:
inventory[inventory['BALANCE_AS_OF_DATE'] == '12/31/2024'].head()

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
14029,12/31/2024,CHINA-WAREHOUSE,MAT-0045,SCRAP,180,KG,232,CNY
14030,12/31/2024,CHINA-WAREHOUSE,MAT-0417,9242A,650,KG,1866,CNY
14031,12/31/2024,CHINA-WAREHOUSE,MAT-0417,5483A,18200,KG,52234,CNY
14032,12/31/2024,CHINA-WAREHOUSE,MAT-0417,4458A,18850,KG,54100,CNY
14033,12/31/2024,CHINA-WAREHOUSE,MAT-0128,SCRAP,14,KG,51,CNY


In [260]:
inventory.head(10)

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
0,12/31/2023,CHINA-WAREHOUSE,MAT-0045,SCRAP,164,KG,211,CNY
1,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6024,0,KG,87666,CNY
2,12/31/2023,CHINA-WAREHOUSE,MAT-0193,5755,70720,KG,414419,CNY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY
4,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6734,2720,KG,15939,CNY
5,12/31/2023,CHINA-WAREHOUSE,MAT-0311,SCRAP,4142,KG,25018,CNY
6,12/31/2023,CHINA-WAREHOUSE,MAT-0374,5995C5416,1000,KG,6430,CNY
7,12/31/2023,CHINA-WAREHOUSE,MAT-0374,8466C2971,15125,KG,97254,CNY
8,12/31/2023,CHINA-WAREHOUSE,MAT-0374,8143C2373,32975,KG,212029,CNY
9,12/31/2023,CHINA-WAREHOUSE,MAT-0046,9532,15125,KG,97254,CNY


In [261]:
inventory[inventory['BATCH_NUMBER'] == '3142']

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY


In [262]:
inventory_grouped = inventory.groupby(['MATERIAL_NAME']).agg(
    total_quantity=('UNRESRICTED_STOCK', 'sum'),
    total_cost=('STOCK_SELL_VALUE', 'sum')
).reset_index().sort_values(by='total_quantity', ascending=True)

inventory_grouped[inventory_grouped['MATERIAL_NAME'] == 'MAT-0045']

,MATERIAL_NAME,total_quantity,total_cost
44,MAT-0045,2826,3097


In [263]:
inventory[inventory['MATERIAL_NAME'] == 'MAT-0331']

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
639,12/31/2023,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,70,CNY
1454,1/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
2314,2/29/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
3213,3/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
4134,4/30/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
5080,5/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
6278,6/30/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
7924,7/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
9387,8/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,70,CNY


In [264]:
# Scraps
inventory_scrap = inventory[inventory['BATCH_NUMBER'] == 'SCRAP']


inventory_scrap_merged = inventory_scrap.merge(
    material[['MATERIAL_NAME', 'SHELF_LIFE_IN_MONTH']],
    on='MATERIAL_NAME',
    how='left'
).drop(columns=['STOCK_UNIT', 'CURRENCY', 'BATCH_NUMBER'])

inventory_scrap_merged.sort_values(by='BALANCE_AS_OF_DATE')

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,UNRESRICTED_STOCK,STOCK_SELL_VALUE,SHELF_LIFE_IN_MONTH
32,1/31/2024,CHINA-WAREHOUSE,MAT-0279,1654,3424,3
22,1/31/2024,CHINA-WAREHOUSE,MAT-0257,5164,10482,8
23,1/31/2024,CHINA-WAREHOUSE,MAT-0258,11187,65558,9
24,1/31/2024,CHINA-WAREHOUSE,MAT-0100,5143,31063,5
25,1/31/2024,CHINA-WAREHOUSE,MAT-0095,1685,2325,9
...,...,...,...,...,...,...
208,9/30/2024,CHINA-WAREHOUSE,MAT-0258,529,3098,9
206,9/30/2024,CHINA-WAREHOUSE,MAT-0194,38,118,2
205,9/30/2024,CHINA-WAREHOUSE,MAT-0045,68,88,10
210,9/30/2024,CHINA-WAREHOUSE,MAT-0095,157,217,9


In [265]:
inventory_scrap_merged

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,UNRESRICTED_STOCK,STOCK_SELL_VALUE,SHELF_LIFE_IN_MONTH
0,12/31/2023,CHINA-WAREHOUSE,MAT-0045,164,211,10
1,12/31/2023,CHINA-WAREHOUSE,MAT-0311,4142,25018,7
2,12/31/2023,CHINA-WAREHOUSE,MAT-0194,100,314,2
3,12/31/2023,CHINA-WAREHOUSE,MAT-0257,5151,10457,8
4,12/31/2023,CHINA-WAREHOUSE,MAT-0258,10048,58879,9
...,...,...,...,...,...,...
257,12/31/2024,CHINA-WAREHOUSE,MAT-0187,544,506,5
258,12/31/2024,CHINA-WAREHOUSE,MAT-0214,535,3439,6
259,12/31/2024,CHINA-WAREHOUSE,MAT-0300,1300,4082,7
260,12/31/2024,CHINA-WAREHOUSE,MAT-0280,11,72,9


In [266]:
temp = inventory_scrap_merged.groupby(['MATERIAL_NAME']).agg(
    total_quantity=('UNRESRICTED_STOCK', 'sum'),
    total_cost=('STOCK_SELL_VALUE', 'sum'),
    shelf_life=('SHELF_LIFE_IN_MONTH', 'first'),
).reset_index().sort_values(by='total_quantity', ascending=True)

temp.head()

,MATERIAL_NAME,total_quantity,total_cost,shelf_life
17,MAT-0219,20,11,6
27,MAT-0331,108,140,6
2,MAT-0036,130,726,5
6,MAT-0078,152,512,3
0,MAT-0010,495,3172,5


## Inbounds grouped by date, material, plant, with the sum

In [267]:
inbound_grouped = inbound.groupby(['INBOUND_DATE', 'MATERIAL_NAME', 'PLANT_NAME']).agg(
    total_quantity=('NET_QUANTITY_MT', 'sum')
).reset_index().sort_values(by='INBOUND_DATE')

inbound_grouped[inbound_grouped['MATERIAL_NAME'] == 'MAT-0339'].head(10)

,INBOUND_DATE,MATERIAL_NAME,PLANT_NAME,total_quantity
215,2024/01/11,MAT-0339,CHINA-WAREHOUSE,25.5
629,2024/03/21,MAT-0339,CHINA-WAREHOUSE,25.5
901,2024/04/29,MAT-0339,CHINA-WAREHOUSE,25.5
1349,2024/07/15,MAT-0339,CHINA-WAREHOUSE,25.5
1421,2024/07/27,MAT-0339,CHINA-WAREHOUSE,0.5
1456,2024/08/01,MAT-0339,CHINA-WAREHOUSE,25.5
1974,2024/11/10,MAT-0339,CHINA-WAREHOUSE,25.5
2116,2024/12/22,MAT-0339,CHINA-WAREHOUSE,25.5


## Same with outbounds

In [268]:
# groupe outbound by date and material_name and plant_name, with separated transports, customer name, and total quantity
outbound_grouped = outbound.groupby(['OUTBOUND_DATE', 'MATERIAL_NAME', 'PLANT_NAME', 'MODE_OF_TRANSPORT', 'CUSTOMER_NUMBER']).agg(
    total_quantity=('NET_QUANTITY_MT', 'sum')
).reset_index().sort_values(by='OUTBOUND_DATE')

outbound_grouped[(outbound_grouped['MATERIAL_NAME'] == 'MAT-0050') 
                 & (outbound_grouped['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE')].head(10)

,OUTBOUND_DATE,MATERIAL_NAME,PLANT_NAME,MODE_OF_TRANSPORT,CUSTOMER_NUMBER,total_quantity
0,2023/12/06,MAT-0050,SINGAPORE-WAREHOUSE,Marine,CST-01001,99.00
64,2024/01/03,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,24.75
287,2024/01/08,MAT-0050,SINGAPORE-WAREHOUSE,Marine,CST-01037,24.75
464,2024/01/11,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,49.50
619,2024/01/15,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,24.75
1874,2024/02/25,MAT-0050,SINGAPORE-WAREHOUSE,Marine,CST-01188,24.75
3009,2024/03/25,MAT-0050,SINGAPORE-WAREHOUSE,Marine,CST-01188,24.75
3874,2024/04/15,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,24.75
4186,2024/04/22,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,24.75
4324,2024/04/25,MAT-0050,SINGAPORE-WAREHOUSE,Truck,CST-01020,24.75


## TBD. Inventory grouped by items per month (?)

In [269]:
initial_inv = inventory.groupby(['BALANCE_AS_OF_DATE', 'PLANT_NAME', 'MATERIAL_NAME']).agg(
    InitialInv=('UNRESRICTED_STOCK', 'sum')
).reset_index()

# Convert the column to datetime format
initial_inv['BALANCE_AS_OF_DATE'] = pd.to_datetime(initial_inv['BALANCE_AS_OF_DATE'], format='%m/%d/%Y')

initial_inv[initial_inv['MATERIAL_NAME'] == 'MAT-0050'].sort_values(by='BALANCE_AS_OF_DATE', ascending=True)

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,InitialInv
1180,2023-12-31,CHINA-WAREHOUSE,MAT-0050,224200
1359,2023-12-31,SINGAPORE-WAREHOUSE,MAT-0050,453750
21,2024-01-31,CHINA-WAREHOUSE,MAT-0050,224100
198,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0050,330000
2129,2024-02-29,SINGAPORE-WAREHOUSE,MAT-0050,305250
1948,2024-02-29,CHINA-WAREHOUSE,MAT-0050,224100
2295,2024-03-31,CHINA-WAREHOUSE,MAT-0050,224100
2486,2024-03-31,SINGAPORE-WAREHOUSE,MAT-0050,558750
2659,2024-04-30,CHINA-WAREHOUSE,MAT-0050,224075
2859,2024-04-30,SINGAPORE-WAREHOUSE,MAT-0050,180375


In [270]:
initial_div_1000 = initial_inv['InitialInv']/1000
initial_inv['InitialInv'] = initial_div_1000

In [271]:
inbound_grouped[inbound_grouped['MATERIAL_NAME'] == 'MAT-0339'].head(10)

,INBOUND_DATE,MATERIAL_NAME,PLANT_NAME,total_quantity
215,2024/01/11,MAT-0339,CHINA-WAREHOUSE,25.5
629,2024/03/21,MAT-0339,CHINA-WAREHOUSE,25.5
901,2024/04/29,MAT-0339,CHINA-WAREHOUSE,25.5
1349,2024/07/15,MAT-0339,CHINA-WAREHOUSE,25.5
1421,2024/07/27,MAT-0339,CHINA-WAREHOUSE,0.5
1456,2024/08/01,MAT-0339,CHINA-WAREHOUSE,25.5
1974,2024/11/10,MAT-0339,CHINA-WAREHOUSE,25.5
2116,2024/12/22,MAT-0339,CHINA-WAREHOUSE,25.5


In [272]:
outbound_grouped[outbound_grouped['MATERIAL_NAME'] == 'MAT-0339'].sort_values(by='OUTBOUND_DATE', ascending=True)

,OUTBOUND_DATE,MATERIAL_NAME,PLANT_NAME,MODE_OF_TRANSPORT,CUSTOMER_NUMBER,total_quantity
54,2024/01/02,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00025,0.35
392,2024/01/09,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00080,0.10
550,2024/01/12,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00025,0.03
813,2024/01/18,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00025,0.10
863,2024/01/19,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00080,0.08
1356,2024/02/01,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00062,1.50
1378,2024/02/02,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00178,24.00
1972,2024/02/27,MAT-0339,SINGAPORE-WAREHOUSE,Marine,CST-01196,22.50
2160,2024/03/03,MAT-0339,SINGAPORE-WAREHOUSE,Marine,CST-01062,4.50
2392,2024/03/07,MAT-0339,CHINA-WAREHOUSE,Truck,CST-00435,0.28


In [273]:
initial_inv[(initial_inv['MATERIAL_NAME'] == 'MAT-0339') & 
            (initial_inv['PLANT_NAME'] == 'CHINA-WAREHOUSE')].sort_values(by='BALANCE_AS_OF_DATE', ascending=True)

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,InitialInv
1312,2023-12-31,CHINA-WAREHOUSE,MAT-0339,1.125
151,2024-01-31,CHINA-WAREHOUSE,MAT-0339,25.975
2077,2024-02-29,CHINA-WAREHOUSE,MAT-0339,0.475
2803,2024-04-30,CHINA-WAREHOUSE,MAT-0339,3.000
3193,2024-05-31,CHINA-WAREHOUSE,MAT-0339,20.725
3595,2024-06-30,CHINA-WAREHOUSE,MAT-0339,14.550
4006,2024-07-31,CHINA-WAREHOUSE,MAT-0339,31.525
4423,2024-08-31,CHINA-WAREHOUSE,MAT-0339,31.525
4833,2024-09-30,CHINA-WAREHOUSE,MAT-0339,16.425
536,2024-10-31,CHINA-WAREHOUSE,MAT-0339,10.075


In [274]:
initial_inv_china = initial_inv[initial_inv['PLANT_NAME'] == 'CHINA-WAREHOUSE']
initial_inv_singapore = initial_inv[initial_inv['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE']

In [275]:
initial_inv_singapore.head(10).sort_values(by='BALANCE_AS_OF_DATE', ascending=True)

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,InitialInv
186,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0005,15.000
187,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0007,5.425
188,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0015,2344.000
189,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0022,6.875
190,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0028,19.250
191,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0029,15.800
192,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0032,25.500
193,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0034,11.000
194,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0035,521.825
195,2024-01-31,SINGAPORE-WAREHOUSE,MAT-0039,19.250


In [276]:
min_date = initial_inv_china['BALANCE_AS_OF_DATE'].min()
max_date = pd.to_datetime("2024-12-31")
full_dates = pd.date_range(min_date, max_date)

# Cross join dates with material list
materials = initial_inv_china['MATERIAL_NAME'].unique()
date_df = pd.DataFrame({'Day': full_dates})
mat_df = pd.DataFrame({'MATERIAL_NAME': materials})
cross = date_df.merge(mat_df, how='cross')

# Merge and forward fill
daily_inv_china = cross.merge(
    initial_inv_china,
    left_on=['Day', 'MATERIAL_NAME'],
    right_on=['BALANCE_AS_OF_DATE', 'MATERIAL_NAME'],
    how='left'
)
daily_inv_china = daily_inv_china.sort_values(['MATERIAL_NAME', 'Day'])
daily_inv_china['InitialInv'] = daily_inv_china.groupby('MATERIAL_NAME')['InitialInv'].ffill()

# Clean up
daily_inv_china = daily_inv_china[['Day', 'MATERIAL_NAME', 'InitialInv']]
daily_inv_china

,Day,MATERIAL_NAME,InitialInv
0,2023-12-31,MAT-0001,1927.525
307,2024-01-01,MAT-0001,1927.525
614,2024-01-02,MAT-0001,1927.525
921,2024-01-03,MAT-0001,1927.525
1228,2024-01-04,MAT-0001,1927.525
...,...,...,...
111401,2024-12-27,MAT-0430,58.450
111708,2024-12-28,MAT-0430,58.450
112015,2024-12-29,MAT-0430,58.450
112322,2024-12-30,MAT-0430,58.450


In [277]:
min_date = initial_inv_china['BALANCE_AS_OF_DATE'].min()
max_date = pd.to_datetime("2024-12-31")
full_dates = pd.date_range(min_date, max_date)

# Cross join dates with material list
materials = initial_inv_singapore['MATERIAL_NAME'].unique()
date_df = pd.DataFrame({'Day': full_dates})
mat_df = pd.DataFrame({'MATERIAL_NAME': materials})
cross = date_df.merge(mat_df, how='cross')

# Merge and forward fill
daily_inv_singapore = cross.merge(
    initial_inv_china,
    left_on=['Day', 'MATERIAL_NAME'],
    right_on=['BALANCE_AS_OF_DATE', 'MATERIAL_NAME'],
    how='left'
)
daily_inv_singapore = daily_inv_singapore.sort_values(['MATERIAL_NAME', 'Day'])
daily_inv_singapore['InitialInv'] = daily_inv_singapore.groupby('MATERIAL_NAME')['InitialInv'].ffill()

# Clean up
daily_inv_singapore = daily_inv_singapore[['Day', 'MATERIAL_NAME', 'InitialInv']]
daily_inv_china

,Day,MATERIAL_NAME,InitialInv
0,2023-12-31,MAT-0001,1927.525
307,2024-01-01,MAT-0001,1927.525
614,2024-01-02,MAT-0001,1927.525
921,2024-01-03,MAT-0001,1927.525
1228,2024-01-04,MAT-0001,1927.525
...,...,...,...
111401,2024-12-27,MAT-0430,58.450
111708,2024-12-28,MAT-0430,58.450
112015,2024-12-29,MAT-0430,58.450
112322,2024-12-30,MAT-0430,58.450


In [278]:
# convert OUTBOUND_DATE to timestamp format
outbound_grouped['OUTBOUND_DATE'] = pd.to_datetime(outbound_grouped['OUTBOUND_DATE'], format='%Y/%m/%d')
outbound_grouped[outbound_grouped['CUSTOMER_NUMBER'] == 'CST-00001'].head(10)

,OUTBOUND_DATE,MATERIAL_NAME,PLANT_NAME,MODE_OF_TRANSPORT,CUSTOMER_NUMBER,total_quantity
15,2024-01-02,MAT-0013,CHINA-WAREHOUSE,Truck,CST-00001,51.0


In [279]:
outbound_grouped_china = outbound_grouped[outbound_grouped['PLANT_NAME'] == 'CHINA-WAREHOUSE']
outbound_grouped_singapore = outbound_grouped[outbound_grouped['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE']
outbound_grouped_china

,OUTBOUND_DATE,MATERIAL_NAME,PLANT_NAME,MODE_OF_TRANSPORT,CUSTOMER_NUMBER,total_quantity
48,2024-01-02,MAT-0317,CHINA-WAREHOUSE,Truck,CST-00020,13.75
39,2024-01-02,MAT-0229,CHINA-WAREHOUSE,Truck,CST-00026,0.10
40,2024-01-02,MAT-0264,CHINA-WAREHOUSE,Truck,CST-00005,99.01
41,2024-01-02,MAT-0266,CHINA-WAREHOUSE,Truck,CST-00019,9.63
42,2024-01-02,MAT-0268,CHINA-WAREHOUSE,Truck,CST-00002,102.00
...,...,...,...,...,...,...
15508,2024-12-31,MAT-0082,CHINA-WAREHOUSE,Truck,CST-00003,7.50
15506,2024-12-31,MAT-0035,CHINA-WAREHOUSE,Truck,CST-00097,49.51
15505,2024-12-31,MAT-0015,CHINA-WAREHOUSE,Truck,CST-00259,24.75
15504,2024-12-31,MAT-0001,CHINA-WAREHOUSE,Truck,CST-00178,51.00


In [280]:
outbound_grouped_china = outbound_grouped_china.groupby(
    ['OUTBOUND_DATE', 'MATERIAL_NAME'], as_index=False
).agg(
    {'total_quantity': 'sum',}
    )

outbound_grouped_china

,OUTBOUND_DATE,MATERIAL_NAME,total_quantity
0,2024-01-02,MAT-0007,24.76
1,2024-01-02,MAT-0013,54.00
2,2024-01-02,MAT-0015,99.00
3,2024-01-02,MAT-0055,12.38
4,2024-01-02,MAT-0056,0.60
...,...,...,...
8249,2024-12-31,MAT-0327,24.75
8250,2024-12-31,MAT-0356,11.00
8251,2024-12-31,MAT-0369,0.03
8252,2024-12-31,MAT-0389,0.03


In [281]:
outbound_grouped_singapore = outbound_grouped_singapore.groupby(
    ['OUTBOUND_DATE', 'MATERIAL_NAME'], as_index=False
).agg(
    {'total_quantity': 'sum',}
    )

outbound_grouped_singapore

,OUTBOUND_DATE,MATERIAL_NAME,total_quantity
0,2023-12-06,MAT-0050,99.000
1,2023-12-31,MAT-0056,49.500
2,2023-12-31,MAT-0210,6.875
3,2023-12-31,MAT-0303,2.500
4,2023-12-31,MAT-0345,1.250
...,...,...,...
3588,2024-12-31,MAT-0398,48.110
3589,2024-12-31,MAT-0426,357.000
3590,2024-12-31,MAT-0428,247.500
3591,2025-01-01,MAT-0172,49.500


In [282]:
# First, rename OUTBOUND_DATE to match the inventory date column
outbound_grouped_china_renamed = outbound_grouped_china.rename(columns={'OUTBOUND_DATE': 'Day'})

# Merge the two DataFrames
merged_inv_china = pd.merge(
    daily_inv_china,
    outbound_grouped_china_renamed,
    how='left',
    on=['Day', 'MATERIAL_NAME']
)

merged_inv_china

,Day,MATERIAL_NAME,InitialInv,total_quantity
0,2023-12-31,MAT-0001,1927.525,NaN
1,2024-01-01,MAT-0001,1927.525,NaN
2,2024-01-02,MAT-0001,1927.525,NaN
3,2024-01-03,MAT-0001,1927.525,25.50
4,2024-01-04,MAT-0001,1927.525,NaN
...,...,...,...,...
112664,2024-12-27,MAT-0430,58.450,NaN
112665,2024-12-28,MAT-0430,58.450,2.76
112666,2024-12-29,MAT-0430,58.450,NaN
112667,2024-12-30,MAT-0430,58.450,NaN


In [283]:
# First, rename OUTBOUND_DATE to match the inventory date column
outbound_grouped_singapore_renamed = outbound_grouped_singapore.rename(columns={'OUTBOUND_DATE': 'Day'})

# Merge the two DataFrames
merged_inv_singapore = pd.merge(
    daily_inv_singapore,
    outbound_grouped_singapore_renamed,
    how='left',
    on=['Day', 'MATERIAL_NAME']
)

merged_inv_singapore

,Day,MATERIAL_NAME,InitialInv,total_quantity
0,2023-12-31,MAT-0005,0.025,NaN
1,2024-01-01,MAT-0005,0.025,NaN
2,2024-01-02,MAT-0005,0.025,NaN
3,2024-01-03,MAT-0005,0.025,NaN
4,2024-01-04,MAT-0005,0.025,NaN
...,...,...,...,...
97984,2024-12-27,MAT-0431,NaN,NaN
97985,2024-12-28,MAT-0431,NaN,NaN
97986,2024-12-29,MAT-0431,NaN,NaN
97987,2024-12-30,MAT-0431,NaN,NaN


In [284]:
def compute_reconstructed_inventory(df):
    df = df.copy()

    # Sort to ensure correct date order
    df = df.sort_values(by=['MATERIAL_NAME', 'Day'])

    # Fill missing total_quantity with 0
    df['total_quantity'] = pd.to_numeric(df['total_quantity'], errors='coerce').fillna(0)

    # Fill missing InitialInv forward (or backward) if needed
    df['InitialInv'] = pd.to_numeric(df['InitialInv'], errors='coerce')
    df['InitialInv'] = df.groupby('MATERIAL_NAME')['InitialInv'].ffill().bfill()

    df['ReconstructedInventory'] = np.nan

    for material, group in df.groupby('MATERIAL_NAME'):
        group = group.copy()
        group = group.sort_values('Day')

        running_inventory = group['InitialInv'].iloc[0]

        for idx in group.index:
            qty = group.at[idx, 'total_quantity']
            running_inventory += qty
            df.at[idx, 'ReconstructedInventory'] = running_inventory

    # Forward fill any remaining NaNs after loop (e.g. for missing rows)
    df['ReconstructedInventory'] = df.groupby('MATERIAL_NAME')['ReconstructedInventory'].ffill()

    return df

In [285]:
cumulative_inv_china = compute_reconstructed_inventory(merged_inv_china)
cumulative_inv_singapore = compute_reconstructed_inventory(merged_inv_singapore)

In [286]:
cumulative_inv_china[cumulative_inv_china['MATERIAL_NAME'] == 'MAT-0001']

,Day,MATERIAL_NAME,InitialInv,total_quantity,ReconstructedInventory
0,2023-12-31,MAT-0001,1927.525,0.0,1927.525
1,2024-01-01,MAT-0001,1927.525,0.0,1927.525
2,2024-01-02,MAT-0001,1927.525,0.0,1927.525
3,2024-01-03,MAT-0001,1927.525,25.5,1953.025
4,2024-01-04,MAT-0001,1927.525,0.0,1953.025
...,...,...,...,...,...
362,2024-12-27,MAT-0001,520.325,25.5,10630.955
363,2024-12-28,MAT-0001,520.325,15.0,10645.955
364,2024-12-29,MAT-0001,520.325,0.0,10645.955
365,2024-12-30,MAT-0001,520.325,51.0,10696.955


In [287]:
cumulative_inv_singapore['ReconstructedInventory'] = cumulative_inv_singapore['ReconstructedInventory'].fillna(0)

In [288]:
cumulative_inv_china.to_csv('../Data/cumulative_inventory_china.csv', index=False)
cumulative_inv_singapore.to_csv('../Data/cumulative_inventory_singapore.csv', index=False)

In [289]:
# group cumulative_inv_chin by material name and date, with the last day reconstructed inventory of the month as that month's and material's entry
cumulative_inv_china['Day'] = pd.to_datetime(cumulative_inv_china['Day'])
cumulative_inv_china['Month'] = cumulative_inv_china['Day'].dt.to_period('M')
cumulative_inv_china = cumulative_inv_china.groupby(['MATERIAL_NAME', 'Month']).agg(
    ReconstructedInventory=('ReconstructedInventory', 'last')
).reset_index()

cumulative_inv_china

,MATERIAL_NAME,Month,ReconstructedInventory
0,MAT-0001,2023-12,1927.525
1,MAT-0001,2024-01,3337.335
2,MAT-0001,2024-02,3719.205
3,MAT-0001,2024-03,4398.995
4,MAT-0001,2024-04,5233.095
...,...,...,...
3986,MAT-0430,2024-08,33.610
3987,MAT-0430,2024-09,38.090
3988,MAT-0430,2024-10,38.450
3989,MAT-0430,2024-11,118.720


In [290]:
cumulative_inv_china[cumulative_inv_china['MATERIAL_NAME'] == 'MAT-0001']

,MATERIAL_NAME,Month,ReconstructedInventory
0,MAT-0001,2023-12,1927.525
1,MAT-0001,2024-01,3337.335
2,MAT-0001,2024-02,3719.205
3,MAT-0001,2024-03,4398.995
4,MAT-0001,2024-04,5233.095
5,MAT-0001,2024-05,5845.325
6,MAT-0001,2024-06,6517.895
7,MAT-0001,2024-07,7303.625
8,MAT-0001,2024-08,7936.975
9,MAT-0001,2024-09,8660.115


In [291]:
import pandas as pd
import pulp
import calendar

In [296]:
# %% [markdown]
# # Data Cleaning

# %%
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt


# %% [markdown]
# Loading in the Datas

# %%
inbound_df = pd.read_csv("../Data/Inbound.csv")
inventory_df = pd.read_csv("../Data/Inventory.csv")
material_master_df = pd.read_csv("../Data/MaterialMaster.csv")
outbound_df = pd.read_csv("../Data/Outbound.csv")
operations_df = pd.read_csv("../Data/OperationCost.csv")

# %%
print("Inventory DataFrame:")
print(inventory_df.head())
print("Inventory DataFrame:")
print("Inventory data columns:", inventory_df.columns.tolist())
print(f"Number of inventory data records: {len(inventory_df)}")

# %%
# Display the first few rows of each DataFrame
print("Inbound DataFrame:")
print(inbound_df.head())
print("Inventory DataFrame:")
print("Inbound data columns:", inbound_df.columns.tolist())
print(f"Number of inbound data records: {len(inbound_df)}")

# %%
print("Outbound DataFrame:")
print(outbound_df.head())
print("Outbound data columns:", outbound_df.columns.tolist())
print(f"Number of outbound data records: {len(outbound_df)}")

# %%
print("Material Master DataFrame:")
print(material_master_df.head())
print("Material Master data columns:", material_master_df.columns.tolist())
print(f"Number of Material Master data records: {len(material_master_df)}")

# %% [markdown]
# ## Checking Null Values

# %%
print("Checking null values in Inbound DataFrame:")
print(inbound_df.isnull().sum())
print("Checking null values in Inventory DataFrame:")
print(inventory_df.isnull().sum())
print("Checking null values in Outbound DataFrame:")
print(outbound_df.isnull().sum())
print("Checking null values in Material Master DataFrame:")
print(material_master_df.isnull().sum())

# %%
inventory_df = inventory_df.dropna()
outbound_df = outbound_df.dropna()
print("Checking null values in Inbound DataFrame:")
print(inbound_df.isnull().sum())
print("Checking null values in Inventory DataFrame:")
print(inventory_df.isnull().sum())
print("Checking null values in Outbound DataFrame:")
print(outbound_df.isnull().sum())
print("Checking null values in Material Master DataFrame:")
print(material_master_df.isnull().sum())

# %%
print("\nPotential Outliers in Inventory:")
numeric_cols = inventory_df.select_dtypes(include='number').columns

for col in numeric_cols:
    Q1 = inventory_df[col].quantile(0.15)
    Q3 = inventory_df[col].quantile(0.85)
    IQR = Q3 - Q1

    outliers = inventory_df[
        (inventory_df[col] < Q1 - 1.5 * IQR) |
        (inventory_df[col] > Q3 + 1.5 * IQR)
    ]

    print(f"{col}: {len(outliers)} potential outliers")


# %% [markdown]
# There are outliers, however we'll just think of those as genuine warehouse events for now

# %% [markdown]
# # Inventory Cleaning

# %% [markdown]
# Count number of scraps

# %%
scrap_count = (inventory_df['BATCH_NUMBER'] == 'SCRAP').sum()
print(f"Number of SCRAPS rows: {scrap_count}")

row_count = inventory_df.shape[0]
print(f"Total number of rows: {row_count}")

print("Percent of SCRAPS rows:", (scrap_count / row_count) * 100)

# just the unique currency codes
print(inventory_df['CURRENCY'].unique())

# or, if you also want counts of each
print(inventory_df['CURRENCY'].value_counts())







# %% [markdown]
# Change all currency to USD and then remove the currency column

# %%

fx_rates = {'SGD': 0.74, 'CNY': 0.14, 'USD': 1.00}
# 1) Copy your original
inventory_usd = inventory_df.copy()

# 2) Clean and convert on the copy
inventory_usd['STOCK_SELL_VALUE'] = (
    inventory_usd['STOCK_SELL_VALUE']
      .replace(r'[^\d\.]', '', regex=True)
      .astype(float)
)
inventory_usd['STOCK_SELL_USD'] = (
    inventory_usd['STOCK_SELL_VALUE']
    * inventory_usd['CURRENCY'].map(fx_rates)
)

inventory_usd.drop(columns=['CURRENCY', 'STOCK_SELL_VALUE'], inplace=True)
# inventory_df is still the old one; inventory_usd has your changes
inventory_usd



# %% [markdown]
# Inspecting SCRAP

# %%
scrap_df = inventory_df[
    inventory_df['BATCH_NUMBER'] == 'SCRAP'
]
scrap_df

# %%
total_scrap_usd = inventory_usd[inventory_usd['BATCH_NUMBER'] == 'SCRAP']['STOCK_SELL_USD'].sum()

print(f"Total SCRAP sell value (USD): {total_scrap_usd:,.2f}")

total_not_scrap_usd = inventory_usd[inventory_usd['BATCH_NUMBER'] != 'SCRAP']['STOCK_SELL_USD'].sum()
print(f"Total non-SCRAP sell value (USD): {total_not_scrap_usd:,.2f}")

# Percentage of total inventory value that is SCRAP
scrap_percentage = (total_scrap_usd / (total_scrap_usd + total_not_scrap_usd) * 100) if (total_scrap_usd + total_not_scrap_usd) else 0
print(f"Percentage of total inventory value that is SCRAP: {scrap_percentage:.2f}%")




# %% [markdown]
# Since it's only 0.03 percent of the total value we can drop all rows with SCRAP

# %%
inventory_cleaned_usd = inventory_usd[
    inventory_usd['BATCH_NUMBER'] != 'SCRAP'
].copy()

inventory_cleaned_usd.reset_index(drop=True, inplace=True)
inventory_cleaned_usd.head()

print("Checking null values in cleaned Inventory DataFrame:")

print("----------------------------")
inventory_cleaned_usd = inventory_cleaned_usd.dropna(
    subset=['BATCH_NUMBER']
).reset_index(drop=True)

print(inventory_cleaned_usd.isnull().sum())


# %% [markdown]
# ## Clean Duplicate Rows

# %%
# duplicates = inventory_cleaned_usd.duplicated()
# print(f"Number of duplicate rows: {duplicates.sum()}")

# print(f"Number of rows before removing duplicates: {inventory_cleaned_usd.shape[0]}")

# inventory_cleaned_usd = inventory_cleaned_usd.drop_duplicates().reset_index(drop=True)
# print(f"Number of rows after removing duplicates: {inventory_cleaned_usd.shape[0]}")

# %%
inventory_cleaned_usd.head()

# Convert BALANCE_AS_OF_DATE to datetime if not already
inventory_cleaned_usd['BALANCE_AS_OF_DATE'] = pd.to_datetime(
    inventory_cleaned_usd['BALANCE_AS_OF_DATE'], errors='coerce'
)

inventory_cleaned_usd['month'] = (
    inventory_cleaned_usd['BALANCE_AS_OF_DATE']
    .dt.to_period('M')
    .astype(str)
)

# Quick check
inventory_cleaned_usd[['BALANCE_AS_OF_DATE','month']].head()


# 1) Keep only Singapore warehouse
sg_inventory = inventory_cleaned_usd[
    inventory_cleaned_usd['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE'
].copy()

# 2) Aggregate unrestricted stock (in MT) by material and month
monthly_stock = (
    sg_inventory
      .groupby(['month','PLANT_NAME','MATERIAL_NAME'], as_index=False)
      .agg(InitialInv_MT=('UNRESRICTED_STOCK','sum'))
      .sort_values(['month','MATERIAL_NAME'])
)

# 3) Inspect the result
monthly_stock.head()

monthly_stock

# %%

forcast = pd.read_csv("../Data/Monthly_Forecast_Summary.csv")
# 2) Prepare inbound events
sg_inb = (
    inbound_df[inbound_df['PLANT_NAME']=='SINGAPORE-WAREHOUSE']
    .assign(
        INBOUND_DATE=lambda df: pd.to_datetime(
            df['INBOUND_DATE'], format='%Y/%m/%d', errors='coerce'
        )
    )
    .sort_values(['MATERIAL_NAME','INBOUND_DATE'])
)

sg_inb.head()

sg_inb_daily = (
    sg_inb
    .groupby(['INBOUND_DATE','MATERIAL_NAME'], as_index=False)
    .agg(InboundQty_MT=('NET_QUANTITY_MT','sum'))
    .sort_values(['INBOUND_DATE','MATERIAL_NAME'])
)

# ensure INBOUND_DATE is datetime
sg_inb_daily['INBOUND_DATE'] = pd.to_datetime(
    sg_inb_daily['INBOUND_DATE'], errors='coerce'
)

# filter to year 2024, keep daily granularity
sg_inb_daily_2024 = sg_inb_daily[
    sg_inb_daily['INBOUND_DATE'].dt.year == 2024
].copy()

# inspect
sg_inb_daily_2024[sg_inb_daily_2024["MATERIAL_NAME"] == "MAT-0056"]

# %%
# 1) make sure date is datetime
sg_inb_daily['INBOUND_DATE'] = pd.to_datetime(sg_inb_daily['INBOUND_DATE'])

# 2) extract month period
sg_inb_daily['month'] = sg_inb_daily['INBOUND_DATE'].dt.to_period('M')

# 3) aggregate into monthly totals
sg_inb_monthly = (
    sg_inb_daily
      .groupby(['month','MATERIAL_NAME'], as_index=False)
      .agg(MonthlyInbound_MT=('InboundQty_MT','sum'))
      .sort_values(['month','MATERIAL_NAME'])
)

sg_inb_monthly.head()

# %%
# Merge with SHELF_LIFE_IN_MONTH from material_master_df
sg_inb_monthly = sg_inb_monthly.merge(
    material_master_df[['MATERIAL_NAME', 'SHELF_LIFE_IN_MONTH', 'DOWNGRADE_VALUE_LOST_PERCENT']],
    on='MATERIAL_NAME',
    how='left'
)

sg_inb_monthly.head()

# %%
forcast_sg = forcast[forcast['Warehouse'] == 'SINGAPORE'].copy()

# %%
forcast_sg

# %%
# multiply the three KT‐columns by 1 000:
forcast_sg["Total_Cap_MT"] = forcast_sg["Total_Cap_KT"] * 1000
forcast_sg["Predicted_Outbound_MT"] = forcast_sg["Predicted_Outbound_KT"] * 1000
forcast_sg["Predicted_Inventory_MT"] = forcast_sg["Predicted_Inventory_KT"] * 1000

# if you’d rather *replace* the old columns in place and rename:
forcast_sg["Total_Cap_KT"] *= 1000
forcast_sg.drop(columns=['Total_Cap_KT', 'Predicted_Outbound_KT', 'Predicted_Inventory_KT'], inplace=True)

# %%
forcast_sg['Month'] = pd.to_datetime(forcast_sg['Month'], errors='coerce').dt.month
forcast_sg.head()

# %%
forcast_sg['Period'] = pd.PeriodIndex(year=2024, month=forcast_sg['Month'], freq='M')
forcast_sg.rename(columns={
    'Month': 'Months'
}, inplace=True)

# %%
forcast_sg.rename(columns={
    'Period': 'Month'
}, inplace=True)
forcast_sg.head()

# %%
operations_df 

# %%
# Drop the row where Operation is "Inventory Storage per MT per day" and Plant/Mode of Transport is "CHINA WAREHOUSE"
operations_df = operations_df[
	~((operations_df["Operation"] == "Inventory Storage per MT per day") & 
	  (operations_df["Plant/Mode of Transport"] == "CHINA WAREHOUSE"))
]
operations_df

# %%
operations_df["Cost_USD"] = operations_df["Cost"] * operations_df["Currency"].map(fx_rates)

# %%
operations_df


# %%
operations_df.drop(columns=['Cost'], inplace=True)
operations_df.drop(columns=['Currency'], inplace=True)
operations_df

# %%
operations_df

# %%
forcast_sg

# %%
# ensure you have a Period column


# drop anything not in 2024
inb_2024 = sg_inb_monthly[sg_inb_monthly['month'].dt.year == 2024]

# build your inbound lookup
inb_dict = (
    inb_2024
    .set_index(['month','MATERIAL_NAME'])['MonthlyInbound_MT']
    .to_dict()
)


# %%
inb_2024

# %%
# when you load forcast_sg:
if not pd.api.types.is_period_dtype(forcast_sg['Month']):
	forcast_sg['Month'] = pd.to_datetime(forcast_sg['Month']).dt.to_period('M')
T = sorted(forcast_sg['Month'].unique())


# %%
import pandas as pd
import pulp
import calendar

# ——— 1. LOAD YOUR DATAFRAMES ———
# You should already have:
#   inb_2024: columns ['month', 'MATERIAL_NAME', 'MonthlyInbound_MT', ...]
#   forcast_sg: columns ['Month', 'Predicted_Outbound_MT', 'Predicted_Inventory_MT']
#   operations_df: columns ['Plant/Mode of Transport', 'Cost_USD'], with one row for
#       'Truck', one for 'Marine', and one for 'SINGAPORE WAREHOUSE' storage

# Example: rename for ease
inv = forcast_sg.set_index('Month')[['Predicted_Outbound_MT','Predicted_Inventory_MT']]
ops = operations_df.set_index('Plant/Mode of Transport')['Cost_USD'].to_dict()
holding_cost_per_mt_per_day = ops['SINGAPORE WAREHOUSE']
truck_cost_per_container = ops['Truck']
marine_cost_per_container = ops['Marine']
container_capacity = 24.75  # MT per container

# convert container costs to per‐MT
truck_cost = truck_cost_per_container / container_capacity
marine_cost = marine_cost_per_container / container_capacity

months = list(inv.index)   # [Period('2024-01', 'M'), Period('2024-02', 'M'), …]
# compute days in each month
days_in_month = {m: calendar.monthrange(m.year, m.month)[1] for m in months}

# ——— 2. CREATE THE LP ———
prob = pulp.LpProblem("SG_Warehouse_Cost_Minimization", pulp.LpMinimize)

# Decision vars: shipments by mode (MT)
ship = pulp.LpVariable.dicts("Ship",
                             ((m,mode) for m in months for mode in ['Truck','Marine']),
                             lowBound=0,
                             cat='Continuous')

# Inventory at end of month (MT) — we’ll force this = forecast for cost calc, but
# you could also relax if you wanted it endogenous.
I = pulp.LpVariable.dicts("Inv", months, lowBound=0, cat='Continuous')

# ——— 3. OBJECTIVE ———
# transport + holding
transport_cost = pulp.lpSum(
    ship[m,'Truck'] * truck_cost + ship[m,'Marine'] * marine_cost
    for m in months
)
holding_cost  = pulp.lpSum(
    I[m] * holding_cost_per_mt_per_day * days_in_month[m]
    for m in months
)
prob += transport_cost + holding_cost

# ——— 4. CONSTRAINTS ———
# a) monthly flow balance: previous inventory + inbound = shipments + end inventory
#    Here inbound = sum of inb_2024 for that month (aggregate across materials)
monthly_inbound = inb_2024.groupby('month')['MonthlyInbound_MT'].sum().to_dict()

for i, m in enumerate(months):
    prev_inv = I[months[i-1]] if i>0 else 0  # assume zero starting inv
    prob += prev_inv + monthly_inbound[m] == ship[m,'Truck'] + ship[m,'Marine'] + I[m], \
            f"flow_balance_{m}"

# b) meet forecast outbound: shipments >= predicted outbound
for m in months:
    prob += ship[m,'Truck'] + ship[m,'Marine'] >= inv.at[m,'Predicted_Outbound_MT'], \
            f"demand_FCST_{m}"

# c) fix inv to forecast (if you want exactly the forecast inventory)
for m in months:
    prob += I[m] == inv.at[m,'Predicted_Inventory_MT'], f"fix_inv_{m}"

# ——— 5. SOLVE & EXTRACT ———
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# build results dataframe
res = []
for m in months:
    res.append({
        'Month': m,
        'Ship_Truck_MT': ship[m,'Truck'].value(),
        'Ship_Marine_MT': ship[m,'Marine'].value(),
        'EndInv_MT': I[m].value()
    })
results_df = pd.DataFrame(res)
print(results_df)
print(f"Total Cost = {pulp.value(prob.objective):,.2f} USD")


# %%
import pandas as pd
import pulp
import calendar

# ——— 0. YOUR INPUT DATAFRAMES ———
# inb_2024: ['month','MATERIAL_NAME','MonthlyInbound_MT',
#            'SHELF_LIFE_IN_MONTH','DOWNGRADE_VALUE_LOST_PERCENT']
# forcast_sg: ['Month','Predicted_Outbound_MT','Predicted_Inventory_MT']
# operations_df: ['Plant/Mode of Transport','Cost_USD']
#
# And we assume you’ve already filtered to Singapore and to 2024.

# ——— 1. PREPARE LOOKUPS & INDEX SETS ———

# for inb_2024.month
if not pd.api.types.is_period_dtype(inb_2024['month']):
    inb_2024['month'] = (
        pd.to_datetime(inb_2024['month'])
          .dt.to_period('M')
    )
# for forcast_sg.Month
if not pd.api.types.is_period_dtype(forcast_sg['Month']):
    forcast_sg['Month'] = (
        pd.to_datetime(forcast_sg['Month'])
          .dt.to_period('M')
    )


# Cost lookups
ops = operations_df.set_index('Plant/Mode of Transport')['Cost_USD'].to_dict()
h_daily = ops['SINGAPORE WAREHOUSE']          # USD per MT per day
transfer = {
    'Truck': ops['Truck']  / 24.75,            # USD per MT
    'Marine': ops['Marine']/ 24.75
}

# Months (Period) and days in each month
months = sorted(forcast_sg['Month'].unique())
days_in_month = {m: calendar.monthrange(m.year, m.month)[1] for m in months}

# Materials, shelf-life & spoilage %
M = inb_2024['MATERIAL_NAME'].unique()
shelf_life = inb_2024.set_index('MATERIAL_NAME')['SHELF_LIFE_IN_MONTH'].to_dict()
spoil_pct  = inb_2024.set_index('MATERIAL_NAME')['DOWNGRADE_VALUE_LOST_PERCENT'].div(100).to_dict()

# Inbound per (month,material)
inb_lookup = (
    inb_2024
      .set_index(['month','MATERIAL_NAME'])['MonthlyInbound_MT']
      .to_dict()
)

# Forecast outbound demand per month
demand = forcast_sg.set_index('Month')['Predicted_Outbound_MT'].to_dict()

# ——— 2. BUILD THE MODEL ———
prob = pulp.LpProblem("Disaggregated_SG_Spoilage", pulp.LpMinimize)

# Decision vars
# Y[m][k][t] = MT of material m shipped by mode k in month t
Y = {
    (m,k,t): pulp.LpVariable(f"Ship_{m}_{k}_{t}", lowBound=0)
    for m in M for k in transfer for t in months
}
# I[m][t] = end-of-month inventory of m in t
I = {
    (m,t): pulp.LpVariable(f"Inv_{m}_{t}", lowBound=0)
    for m in M for t in months
}
# Z[m][t] = MT of m that spoil (are removed) in t
Z = {
    (m,t): pulp.LpVariable(f"Spoil_{m}_{t}", lowBound=0)
    for m in M for t in months
}

# ——— 3. OBJECTIVE ———
prob += (
    # transport cost
    pulp.lpSum( Y[m,k,t] * transfer[k]
                for m in M for k in transfer for t in months )
    +
    # holding cost
    pulp.lpSum( I[m,t] * h_daily * days_in_month[t]
                for m in M for t in months )
    +
    # spoilage penalty (folding in per‐MT value = 1)
    pulp.lpSum( Z[m,t] * spoil_pct[m]
                for m in M for t in months )
), "Total_Cost"

# ——— 4. CONSTRAINTS ———

# 4A) Inventory balance per material & month
for m in M:
    for i, t in enumerate(months):
        prev_I = I[m, months[i-1]] if i>0 else 0
        inb    = inb_lookup.get((t,m), 0.0)
        shipped= pulp.lpSum(Y[m,k,t] for k in transfer)
        prob += (
            prev_I
            + inb
            - shipped
            - Z[m,t]
            == I[m,t]
        ), f"Balance_{m}_{t}"

# 4B) Shelf-life ⇒ force spoilage of “old” inventory
for m in M:
    s = int(shelf_life[m])
    for i, t in enumerate(months):
        if i >= s:
            old_t = months[i - s]
            # inventory that was alive at old_t must either have shipped or spoiled by t
            prob += I[m, old_t] <= Z[m, t], f"ShelfLife_{m}_{t}"

# 4C) Meet aggregate outbound demand each month
for t in months:
    prob += (
        pulp.lpSum(Y[m,k,t] for m in M for k in transfer)
        >= demand.get(t, 0.0)
    ), f"Demand_{t}"

# ——— 5. SOLVE & REPORT ———
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# Build result tables
ship_rows = []
for m in M:
    for t in months:
        ship_rows.append({
            'Material': m,
            'Month':     t.to_timestamp() if hasattr(t, 'to_timestamp') else t,
            'Ship_Truck': Y[m,'Truck',t].value(),
            'Ship_Marine':Y[m,'Marine',t].value(),
            'Inv_End':    I[m,t].value(),
            'Spoiled':    Z[m,t].value()
        })
results = pd.DataFrame(ship_rows)
print(results)
print("Total cost:", pulp.value(prob.objective))


# %%
results

# %%
import pandas as pd
import pulp
import calendar

# ——— 0. YOUR INPUT DATAFRAMES ———
# inb_2024: ['month','MATERIAL_NAME','MonthlyInbound_MT',
#            'SHELF_LIFE_IN_MONTH','DOWNGRADE_VALUE_LOST_PERCENT']
# forcast_sg: ['Month','Predicted_Outbound_MT','Predicted_Inventory_MT']
# operations_df: ['Plant/Mode of Transport','Cost_USD']
# (filtered to Singapore & 2024)

# ——— 1. PREPARE LOOKUPS & INDEX SETS ———

# Normalize month columns to pandas Period('M')
if not pd.api.types.is_period_dtype(inb_2024['month']):
    inb_2024['month'] = pd.to_datetime(inb_2024['month']).dt.to_period('M')
if not pd.api.types.is_period_dtype(forcast_sg['Month']):
    forcast_sg['Month'] = pd.to_datetime(forcast_sg['Month']).dt.to_period('M')

# Holding cost per MT per day
ops     = operations_df.set_index('Plant/Mode of Transport')['Cost_USD'].to_dict()
h_daily = ops['SINGAPORE WAREHOUSE']

# Index sets
months        = sorted(forcast_sg['Month'].unique())
days_in_month = {m: calendar.monthrange(m.year, m.month)[1] for m in months}
M             = inb_2024['MATERIAL_NAME'].unique()

# Material params
shelf_life = inb_2024.set_index('MATERIAL_NAME')['SHELF_LIFE_IN_MONTH'].to_dict()
spoil_pct  = inb_2024.set_index('MATERIAL_NAME')['DOWNGRADE_VALUE_LOST_PERCENT'].div(100).to_dict()

# Data lookups
inb_lookup = inb_2024.set_index(['month','MATERIAL_NAME'])['MonthlyInbound_MT'].to_dict()
demand     = forcast_sg.set_index('Month')['Predicted_Outbound_MT'].to_dict()

# ——— 2. BUILD THE MODEL ———
prob = pulp.LpProblem("SG_Holding_vs_Spoilage", pulp.LpMinimize)

# Decision variables
# Shipments (still needed for constraints, but cost-free)
Y = { (m, mode, t): pulp.LpVariable(f"Ship_{m}_{mode}_{t}", lowBound=0)
      for m in M for mode in ['Truck','Marine'] for t in months }
# Inventory end-of-month
I = { (m, t): pulp.LpVariable(f"Inv_{m}_{t}", lowBound=0)
      for m in M for t in months }
# Spoilage
Z = { (m, t): pulp.LpVariable(f"Spoil_{m}_{t}", lowBound=0)
      for m in M for t in months }

# ——— 3. OBJECTIVE ———
prob += (
    # holding cost
    pulp.lpSum(
        I[m,t] * h_daily * days_in_month[t]
        for m in M for t in months
    )
    +
    # spoilage penalty
    pulp.lpSum(
        Z[m,t] * spoil_pct[m]
        for m in M for t in months
    )
), "Holding_and_Spoilage_Cost"

# ——— 4. CONSTRAINTS ———

# 4A) Inventory balance
for m in M:
    for i, t in enumerate(months):
        prev_I = I[m, months[i-1]] if i > 0 else 0
        inb    = inb_lookup.get((t,m), 0.0)
        shipped= pulp.lpSum(Y[m,mode,t] for mode in ['Truck','Marine'])
        prob += (
            prev_I + inb - shipped - Z[m,t] == I[m,t]
        ), f"Balance_{m}_{t}"

# 4B) Shelf‐life spoilage
for m in M:
    s = int(shelf_life[m])
    for i, t in enumerate(months):
        if i >= s:
            old_t = months[i - s]
            prob += I[m, old_t] <= Z[m, t], f"ShelfLife_{m}_{t}"

# 4C) Meet outbound demand exactly
for t in months:
    prob += (
        pulp.lpSum(Y[m,mode,t] for m in M for mode in ['Truck','Marine'])
        == demand.get(t, 0.0)
    ), f"Demand_{t}"

# ——— 5. SOLVE & REPORT ———
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# Collect results
rows = []
for m in M:
    for t in months:
        rows.append({
            'Material':    m,
            'Month':       t.to_timestamp(),
            'Ship_Truck':  Y[m,'Truck',t].value(),
            'Ship_Marine': Y[m,'Marine',t].value(),
            'Inv_End':     I[m,t].value(),
            'Spoiled':     Z[m,t].value()
        })
results_df = pd.DataFrame(rows)
print(results_df)
print("Total cost:", pulp.value(prob.objective))


# %%
import pandas as pd
import pulp
import calendar

# ——— 0. YOUR INPUT DATAFRAMES ———
# inb_2024: ['month','MATERIAL_NAME','MonthlyInbound_MT',
#            'SHELF_LIFE_IN_MONTH','DOWNGRADE_VALUE_LOST_PERCENT']
# forcast_sg: ['Month','Total_Cap_MT','Predicted_Outbound_MT','Predicted_Inventory_MT']
# operations_df: ['Plant/Mode of Transport','Cost_USD']
#
# Assume all already filtered to SINGAPORE & 2024.

# ——— 1. PREPARE LOOKUPS & INDEX SETS ———

# a) normalize to Period('M')
if not pd.api.types.is_period_dtype(inb_2024['month']):
    inb_2024['month'] = pd.to_datetime(inb_2024['month']).dt.to_period('M')
if not pd.api.types.is_period_dtype(forcast_sg['Month']):
    forcast_sg['Month'] = pd.to_datetime(forcast_sg['Month']).dt.to_period('M')

# b) build cost lookups (we’ll ignore transport cost in the objective, but still need modes)
ops     = operations_df.set_index('Plant/Mode of Transport')['Cost_USD'].to_dict()
h_daily = ops['SINGAPORE WAREHOUSE']  # USD per MT per day

# c) index sets
months        = sorted(forcast_sg['Month'].unique())
days_in_month = {m: calendar.monthrange(m.year, m.month)[1] for m in months}
M             = inb_2024['MATERIAL_NAME'].unique()
modes         = ['Truck','Marine']

# d) parameters per SKU
shelf_life = inb_2024.set_index('MATERIAL_NAME')['SHELF_LIFE_IN_MONTH'].to_dict()
spoil_pct  = inb_2024.set_index('MATERIAL_NAME')['DOWNGRADE_VALUE_LOST_PERCENT'].div(100).to_dict()

# e) inbound & forecast lookups
inb_lookup = inb_2024.set_index(['month','MATERIAL_NAME'])['MonthlyInbound_MT'].to_dict()
demand     = forcast_sg.set_index('Month')['Predicted_Outbound_MT'].to_dict()
capacity   = forcast_sg.set_index('Month')['Total_Cap_MT'].to_dict()

# ——— 2. BUILD THE LP ———
prob = pulp.LpProblem("SG_Monthly_Summary_with_Capacity", pulp.LpMinimize)

# — Decision vars — 
# Y[m,k,t]: MT of material m shipped by mode k in month t
Y = {
    (m,mode,t): pulp.LpVariable(
        name=f"Ship_{m}_{mode}_{t}",
        lowBound=0,
        cat='Continuous'
    )
    for m in M for mode in ['Truck','Marine'] for t in months
}

# I[m,t]: end-of-month inventory of m in t
I = { (m,t):    pulp.LpVariable(f"Inv_{m}_{t}", lowBound=0)
      for m in M for t in months }
# Z[m,t]: MT of m that spoil in t
Z = { (m,t):    pulp.LpVariable(f"Spoil_{m}_{t}", lowBound=0)
      for m in M for t in months }

# — Objective: only holding + spoilage cost ——
prob += (
    pulp.lpSum(I[m,t] * h_daily * days_in_month[t]
               for m in M for t in months)
  + pulp.lpSum(Z[m,t] * spoil_pct[m]
               for m in M for t in months)
), "Holding_and_Spoilage_Cost"

# ——— 3. CONSTRAINTS ———

# 3A) Inventory balance per SKU
# 4D) Supply‐cap: for each SKU & month, shipments ≤ available
for m in M:
    for i, t in enumerate(months):
        # previous-ending-inventory variable (0 if t is first month)
        prev_I = I[m, months[i-1]] if i>0 else 0
        inb    = inb_lookup.get((t,m), 0.0)
        # total shipped of SKU m in t
        total_ship_m_t = pulp.lpSum(Y[m,mode,t] for mode in ['Truck','Marine'])
        # cap shipment by what you actually have
        prob += total_ship_m_t <= prev_I + inb, f"SupplyCap_{m}_{t}"


# 3B) Shelf-life ⇒ spoilage
for m in M:
    s = int(shelf_life[m])
    for i, t in enumerate(months):
        if i >= s:
            old_t = months[i - s]
            prob += I[m, old_t] <= Z[m, t], f"Shelf_{m}_{t}"

# 3C) Meet monthly outbound demand
for t in months:
    prob += (
      pulp.lpSum(Y[m,mode,t] for m in M for mode in ['Truck','Marine'])
      == demand[t]
    ), f"Demand_{t}"

# 3D) Warehouse capacity: total end-inventory ≤ capacity
for t in months:
    prob += (
        pulp.lpSum(I[m,t] for m in M)
        <= capacity[t]
    ), f"Cap_{t}"

# ——— 4. SOLVE ———
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# ——— 5. BUILD MONTHLY SUMMARY ———
rows = []
for t in months:
    total_truck  = sum(Y[m,'Truck',t].value() for m in M)
    total_marine = sum(Y[m,'Marine',t].value() for m in M)
    end_inv      = sum(I[m,t].value()            for m in M)
    rows.append({
        'Month':         t.to_timestamp(),
        'Ship_Truck_MT': total_truck,
        'Ship_Marine_MT':total_marine,
        'EndInv_MT':     end_inv
    })

monthly_summary = pd.DataFrame(rows)
print(monthly_summary)
print("Total cost:", pulp.value(prob.objective))


# %%
operations_df




Inventory DataFrame:
  BALANCE_AS_OF_DATE       PLANT_NAME MATERIAL_NAME BATCH_NUMBER  \
0         12/31/2023  CHINA-WAREHOUSE      MAT-0045        SCRAP   
1         12/31/2023  CHINA-WAREHOUSE      MAT-0193         6024   
2         12/31/2023  CHINA-WAREHOUSE      MAT-0193         5755   
3         12/31/2023  CHINA-WAREHOUSE      MAT-0193         3142   
4         12/31/2023  CHINA-WAREHOUSE      MAT-0193         6734   

   UNRESRICTED_STOCK STOCK_UNIT  STOCK_SELL_VALUE CURRENCY  
0                164         KG               211      CNY  
1                  0         KG             87666      CNY  
2              70720         KG            414419      CNY  
3              12240         KG             71726      CNY  
4               2720         KG             15939      CNY  
Inventory DataFrame:
Inventory data columns: ['BALANCE_AS_OF_DATE', 'PLANT_NAME', 'MATERIAL_NAME', 'BATCH_NUMBER', 'UNRESRICTED_STOCK', 'STOCK_UNIT', 'STOCK_SELL_VALUE', 'CURRENCY']
Number of inventory da

/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:323: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  forcast_sg['Period'] = pd.PeriodIndex(year=2024, month=forcast_sg['Month'], freq='M')
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:383: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(forcast_sg['Month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:493: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(inb_2024['month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:499: DeprecationWarning: is_period_dtype is deprecated and will be removed in a futur

      Material      Month  Ship_Truck  Ship_Marine  Inv_End  Spoiled
0     MAT-0037 2024-01-01         0.0         0.00      0.0      0.0
1     MAT-0037 2024-02-01         0.0         0.00      0.0      0.0
2     MAT-0037 2024-03-01         0.0         0.00      0.0      0.0
3     MAT-0037 2024-04-01         0.0       222.75      0.0      0.0
4     MAT-0037 2024-05-01         0.0         0.00      0.0      0.0
...        ...        ...         ...          ...      ...      ...
1459  MAT-0140 2024-08-01         0.0         0.00      0.0      0.0
1460  MAT-0140 2024-09-01         0.0         0.00      0.0      0.0
1461  MAT-0140 2024-10-01         0.0         0.00      0.0      0.0
1462  MAT-0140 2024-11-01         0.0       123.75      0.0      0.0
1463  MAT-0140 2024-12-01         0.0         0.00      0.0      0.0

[1464 rows x 6 columns]
Total cost: 111606106.51212123
      Material      Month  Ship_Truck  Ship_Marine  Inv_End  Spoiled
0     MAT-0037 2024-01-01       24.75         0

/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:638: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(inb_2024['month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:640: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(forcast_sg['Month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:752: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(inb_2024['month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/621110790.py:754: DeprecationWarning: is_period_dtype is deprecated and will be removed in a 

        Month  Ship_Truck_MT  Ship_Marine_MT  EndInv_MT
0  2024-01-01       5732.500        2977.500   3821.005
1  2024-02-01       8191.578        2758.422      0.000
2  2024-03-01       3741.000        5619.000      0.000
3  2024-04-01       4786.375        2733.625      0.000
4  2024-05-01       3662.875        7077.125   3206.500
5  2024-06-01       5715.250        3724.750   2721.000
6  2024-07-01       4832.250        6547.750      0.000
7  2024-08-01      10936.750         393.250   6189.375
8  2024-09-01      11639.875        5060.125  11379.250
9  2024-10-01      17341.750        3638.250  12125.250
10 2024-11-01      14773.500        1696.500  13430.000
11 2024-12-01      13970.000           0.000      0.000
Total cost: 17887991.23325


,Operation,Plant/Mode of Transport,Cost_USD
0,Inventory Storage per MT per day,SINGAPORE WAREHOUSE,11.1
2,Transfer cost per container (24.75MT),Truck,111.0
3,Transfer cost per container (24.75MT),Marine,75.0


In [298]:
cumulative_inv_singapore_monthly.to_csv('../Data/cumulative_inventory_singapore_monthly.csv', index=False)
cumulative_inv_china_monthly.to_csv('../Data/cumulative_inventory_china_monthly.csv', index=False)

In [300]:
# combine the cumulative intventory for singapore and china
cumulative_inv_monthly = pd.concat([cumulative_inv_singapore_monthly, cumulative_inv_china_monthly], ignore_index=True)
cumulative_inv_monthly.to_csv('../Data/cumulative_inventory_monthly.csv', index=False)

In [303]:
import pandas as pd
import pulp
import calendar

# ——— 0. YOUR INPUT DATAFRAMES ———
# inb_2024: ['month','MATERIAL_NAME','MonthlyInbound_MT',
#            'SHELF_LIFE_IN_MONTH','DOWNGRADE_VALUE_LOST_PERCENT']
# forcast_sg: ['Month','Total_Cap_MT','Predicted_Outbound_MT','Predicted_Inventory_MT']
# operations_df: ['Plant/Mode of Transport','Cost_USD']
# NEW: cumulative_inv_singapore_monthly: ['MATERIAL_NAME','Month','ReconstructedInventory']
#
# Assume all already filtered to SINGAPORE & 2024.

# Load cumulative inventor

# ——— 1. PREPARE LOOKUPS & INDEX SETS ———

# a) Normalize to Period('M')
if not pd.api.types.is_period_dtype(inb_2024['month']):
    inb_2024['month'] = pd.to_datetime(inb_2024['month']).dt.to_period('M')
if not pd.api.types.is_period_dtype(forcast_sg['Month']):
    forcast_sg['Month'] = pd.to_datetime(forcast_sg['Month']).dt.to_period('M')

# b) Build cost lookups
ops = operations_df.set_index('Plant/Mode of Transport')['Cost_USD'].to_dict()
h_daily = ops['SINGAPORE WAREHOUSE']  # USD per MT per day

# c) Index sets
months = sorted(forcast_sg['Month'].unique())
days_in_month = {m: calendar.monthrange(m.year, m.month)[1] for m in months}
M = inb_2024['MATERIAL_NAME'].unique()
modes = ['Truck', 'Marine']

# d) Parameters per SKU
shelf_life = inb_2024.set_index('MATERIAL_NAME')['SHELF_LIFE_IN_MONTH'].to_dict()
spoil_pct = inb_2024.set_index('MATERIAL_NAME')['DOWNGRADE_VALUE_LOST_PERCENT'].div(100).to_dict()

# e) Inbound & forecast lookups
inb_lookup = inb_2024.set_index(['month', 'MATERIAL_NAME'])['MonthlyInbound_MT'].to_dict()
demand = forcast_sg.set_index('Month')['Predicted_Outbound_MT'].to_dict()
capacity = forcast_sg.set_index('Month')['Total_Cap_MT'].to_dict()

# f) Initial inventory from cumulative monthly Singapore
initial_inv_month = pd.Period('2023-12') if pd.Period('2023-12') in cumulative_inv_monthly['Month'].values else pd.Period('2024-01')
initial_inv = (cumulative_inv_monthly[
    cumulative_inv_monthly['Month'] == initial_inv_month
].set_index('MATERIAL_NAME')['ReconstructedInventory'].to_dict())

# ——— 2. BUILD THE LP ———
prob = pulp.LpProblem("SG_Monthly_Summary_with_Capacity", pulp.LpMinimize)

# Decision vars
Y = {
    (m, mode, t): pulp.LpVariable(f"Ship_{m}_{mode}_{t}", lowBound=0, cat='Continuous')
    for m in M for mode in modes for t in months
}
I = {(m, t): pulp.LpVariable(f"Inv_{m}_{t}", lowBound=0) for m in M for t in months}
Z = {(m, t): pulp.LpVariable(f"Spoil_{m}_{t}", lowBound=0) for m in M for t in months}

# Objective: holding + spoilage cost
prob += (
    pulp.lpSum(I[m, t] * h_daily * days_in_month[t] for m in M for t in months) +
    pulp.lpSum(Z[m, t] * spoil_pct[m] for m in M for t in months)
), "Holding_and_Spoilage_Cost"

# ——— 3. CONSTRAINTS ———

# 3A) Inventory balance per SKU
for m in M:
    for i, t in enumerate(months):
        if i == 0:
            prev_I = initial_inv.get(m, 0.0)
        else:
            prev_I = I[m, months[i-1]]
        inb = inb_lookup.get((t, m), 0.0)
        total_ship_m_t = pulp.lpSum(Y[m, mode, t] for mode in modes)
        prob += I[m, t] == prev_I + inb - total_ship_m_t, f"InvBalance_{m}_{t}"
        prob += total_ship_m_t <= prev_I + inb, f"SupplyCap_{m}_{t}"

# 3B) Spoilage based on shelf-life
for m in M:
    s = int(shelf_life[m])
    for i, t in enumerate(months):
        if i >= s:
            old_t = months[i - s]
            prob += I[m, old_t] <= Z[m, t], f"Shelf_{m}_{t}"

# 3C) Fulfill monthly demand
for t in months:
    prob += (
        pulp.lpSum(Y[m, mode, t] for m in M for mode in modes) == demand[t]
    ), f"Demand_{t}"

# 3D) Warehouse capacity
for t in months:
    prob += (
        pulp.lpSum(I[m, t] for m in M) <= capacity[t]
    ), f"Cap_{t}"

# ——— 4. SOLVE ———
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# ——— 5. MONTHLY SUMMARY ———
rows = []
for t in months:
    total_truck = sum(Y[m, 'Truck', t].value() for m in M)
    total_marine = sum(Y[m, 'Marine', t].value() for m in M)
    end_inv = sum(I[m, t].value() for m in M)
    rows.append({
        'Month': t.to_timestamp(),
        'Ship_Truck_MT': total_truck,
        'Ship_Marine_MT': total_marine,
        'EndInv_MT': end_inv
    })

monthly_summary = pd.DataFrame(rows)
print(monthly_summary)
print("Total cost:", pulp.value(prob.objective))


        Month  Ship_Truck_MT  Ship_Marine_MT     EndInv_MT
0  2024-01-01    5732.300000     2977.700000   8003.534100
1  2024-02-01    9058.028004     1891.972003   7827.573103
2  2024-03-01    6676.299991     2683.700001  16238.624111
3  2024-04-01    3895.299997     3624.699996  24547.549133
4  2024-05-01    5791.828956     4948.171026  27481.124251
5  2024-06-01   -2002.308099    11442.308104  26313.524310
6  2024-07-01    8139.775025     3240.224991  26763.449420
7  2024-08-01    8486.127977     2843.872020  34222.899423
8  2024-09-01   12823.799994     3876.200025  30366.549620
9  2024-10-01    6657.294584    14322.705389  21463.649713
10 2024-11-01   11126.674607     5343.325350  11316.599860
11 2024-12-01    8382.999837     5587.000157      0.000000
Total cost: 79511088.49002346


/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/3141963174.py:19: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(inb_2024['month']):
/var/folders/31/0q4j2qxx7yb_s7468v3hrsd40000gn/T/ipykernel_95617/3141963174.py:21: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if not pd.api.types.is_period_dtype(forcast_sg['Month']):


In [306]:
inventory_singapore = inventory[inventory['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE'].copy()

inventory_singapore

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
15510,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4720,2400,KG,1680,SGD
15511,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,8259,65925,KG,46148,SGD
15512,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4874,21975,KG,15383,SGD
15513,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4382,23225,KG,16258,SGD
15514,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,6631,71150,KG,49805,SGD
...,...,...,...,...,...,...,...,...
28323,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0172,M2299A,74250,KG,20790,SGD
28324,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0172,M3493A,15125,KG,4235,SGD
28325,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0191,K1286,325,KG,617500,SGD
28326,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0191,K8761,3000,KG,5700000,SGD


In [308]:

# remove rows with 0 as unrestricted stock, if there are no other rows of that material
inventory_singapore = inventory_singapore[inventory_singapore.groupby('MATERIAL_NAME')['UNRESRICTED_STOCK'].transform('count') > 1].copy()
inventory_singapore

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
15510,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4720,2400,KG,1680,SGD
15511,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,8259,65925,KG,46148,SGD
15512,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4874,21975,KG,15383,SGD
15513,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,4382,23225,KG,16258,SGD
15514,12/31/2023,SINGAPORE-WAREHOUSE,MAT-0130,6631,71150,KG,49805,SGD
...,...,...,...,...,...,...,...,...
28323,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0172,M2299A,74250,KG,20790,SGD
28324,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0172,M3493A,15125,KG,4235,SGD
28325,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0191,K1286,325,KG,617500,SGD
28326,12/31/2024,SINGAPORE-WAREHOUSE,MAT-0191,K8761,3000,KG,5700000,SGD


In [ ]:
# find the average cost per KG of each material (beware of outliers)
inventory_singapore['COST_PER_KG'] = inventory_singapore['STOCK_SELL_VALUE'] / inventory_singapore['UNRESRICTED_STOCK']
average_cost_per_kg = inventory_singapore.groupby('MATERIAL_NAME')['COST_PER_KG'].mean().reset_index()
average_cost_per_kg.rename(columns={'COST_PER_KG': 'AVG_COST_PER_KG'}, inplace=True)   
# merge the average cost per kg back to the inventory dataframe
inventory_singapore = inventory_singapore.merge(average_cost_per_kg, on='MATERIAL_NAME', how='left')
# calculate the cost of the unrestricted stock in USD
inventory_singapore['COST_USD'] = inventory_singapore['UNRESRICTED_STOCK'] * inventory_singapore['AVG_COST_PER_KG']


KeyError: 'COST_USD'